#### Import all the required Libraries

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Load the trained model and Haarcascades

In [2]:
face_classifier = cv2.CascadeClassifier("E:\Desktop_C\practice\Deep Learning\Face Mask Detection\haarcascade_frontalface_default.xml")
model_path = "E:\Desktop_C\practice\Deep Learning\Face Mask Detection\model_face_mask_detection.h5"
model = load_model(model_path)

labels_dict = {0: 'MASK IS ON', 1: 'PLEASE WEAR YOUR MASK'}
color_dict = {0: (0,255,0), 1:(0,0,255)}

In [3]:
SIZE = 4

In [5]:
#setting the source as webcam
cap = cv2.VideoCapture(0)
while(True):
    #reading the image frames
    ret,img = cap.read()
    #converting to grayscale images
    img = cv2.flip(img,1,1) #Flip to act as mirror
    
    #Resize img to speed up detection
    img_1 = cv2.resize(img, (img.shape[1]//SIZE, img.shape[0]//SIZE))
    
    #multiple faces
    faces = face_classifier.detectMultiScale(img_1)  
    
       
    for f in faces:
        #Drawing Rectangle on face
        (x,y,w,h) = [v * SIZE for v in f] #scaleup the size
        
        #Extracting ROI (Bounding Boxes) of the face from the image
        roi_img = img[y:y+h, x:x+w]
        roi_img = cv2.resize(roi_img, (224,224))
        
        #If there is atleast one face in frame
        if np.sum(roi_img)!= 0: 
            roi = roi_img.astype('float')/255
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis =0)        
        
            #Predicting the mask 
            pred = model.predict(roi)[0]
            label = np.argmax(pred)
            
            cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
            cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)

            #putting text on face rectangle drawn above
            cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
               
        else:
            cv2.putText(gray, "no face", (20,60),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)
        
        # Show the image
        cv2.imshow("LIVE", img) 
   
    #Press q to exit the video stream
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
# Stop video
cap.release()


# Close all started windows
cv2.destroyAllWindows()